In [118]:
import re
from google_drive_downloader import GoogleDriveDownloader as gdd

In [119]:
gdd.download_file_from_google_drive(file_id="1moAgo9pulaAMUCTvcqjdNAefyL-I35o0",
                                    dest_path="./train_data.txt",
                                    unzip=False)

In [120]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [121]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [122]:
len(text)

205

In [123]:
num_characters = len(set(text)) #уникальные символы
num_characters                  #len(np.unique(np.array([x for x in text])))

30

In [124]:
import keras
from keras.preprocessing.text import Tokenizer

In [125]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [126]:
tokenizer.fit_on_texts(text)

In [127]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [128]:
inp_chars = 6 #кол-во вводимых символов
data = tokenizer.texts_to_matrix(text)

In [129]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [130]:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

199

In [131]:
import numpy as np

In [132]:
# генерация обучающего набора
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [133]:
len(X), len(Y)

(199, 199)

In [134]:
X[0]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [135]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [136]:
from keras.layers import *
from keras.models import Sequential

In [137]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, 500)               265500    
                                                                 
 dense_6 (Dense)             (None, 30)                15030     
                                                                 
Total params: 280530 (1.07 MB)
Trainable params: 280530 (1.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [138]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 [==============================] - 1s 14ms/step - loss: 3.2944 - accuracy: 0.1407
Epoch 2/100
7/7 [==============================] - 0s 13ms/step - loss: 2.3142 - accuracy: 0.3116
Epoch 3/100
7/7 [==============================] - 0s 14ms/step - loss: 1.8327 - accuracy: 0.4925
Epoch 4/100
7/7 [==============================] - 0s 14ms/step - loss: 1.4320 - accuracy: 0.5829
Epoch 5/100
7/7 [==============================] - 0s 14ms/step - loss: 1.1957 - accuracy: 0.6332
Epoch 6/100
7/7 [==============================] - 0s 14ms/step - loss: 1.0009 - accuracy: 0.6985
Epoch 7/100
7/7 [==============================] - 0s 14ms/step - loss: 0.8219 - accuracy: 0.8040
Epoch 8/100
7/7 [==============================] - 0s 13ms/step - loss: 0.7381 - accuracy: 0.8291
Epoch 9/100
7/7 [==============================] - 0s 16ms/step - loss: 0.5832 - accuracy: 0.9045
Epoch 10/100
7/7 [==============================] - 0s 14ms/step - loss: 0.6191 - accuracy: 0.8342
Epoch 11/100
7/7 [=

In [139]:
def buildPhrase(inp_str, str_len = 1):
  _inp_chars = inp_chars if len(inp_str) > inp_chars else len(inp_str)

  for i in range(str_len):
    x = []
    for j in range(len(inp_str) - _inp_chars, len(inp_str)):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)

    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp, verbose = "None" ) # предсказываем OHE седьмого символа
    char = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += char # дописываем строку

  return inp_str

In [140]:
print(buildPhrase("позитивн", 10))
print(buildPhrase("способ", 50))
print(buildPhrase("проблемы", 76))

позитивно и верьте
способность достигать отличных результатовесли вы смпгли
проблемы которые возникли в понедельникдумайте позитивно и верьте в свою способность


# Слова


In [141]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [142]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [143]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [144]:
data = tokenizer.texts_to_sequences([texts])

In [145]:
data

[[1,
  4,
  5,
  6,
  7,
  8,
  9,
  2,
  3,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  16,
  17,
  18,
  19,
  1,
  20,
  2,
  3,
  21,
  22,
  23,
  24,
  1,
  25,
  26]]

In [146]:
len(data[0])

31

In [147]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [148]:
inp_words = 3
n = res.shape[0]-inp_words

In [149]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [150]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_7 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_7 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [151]:
history = model.fit(X, Y, batch_size=32, epochs=19)

Epoch 1/19
1/1 [==============================] - 2s 2s/step - loss: 6.9046 - accuracy: 0.0000e+00
Epoch 2/19
1/1 [==============================] - 0s 23ms/step - loss: 6.8788 - accuracy: 0.0000e+00
Epoch 3/19
1/1 [==============================] - 0s 26ms/step - loss: 6.8527 - accuracy: 0.1071
Epoch 4/19
1/1 [==============================] - 0s 32ms/step - loss: 6.8262 - accuracy: 0.3571
Epoch 5/19
1/1 [==============================] - 0s 27ms/step - loss: 6.7991 - accuracy: 0.5357
Epoch 6/19
1/1 [==============================] - 0s 22ms/step - loss: 6.7709 - accuracy: 0.6786
Epoch 7/19
1/1 [==============================] - 0s 21ms/step - loss: 6.7415 - accuracy: 0.7500
Epoch 8/19
1/1 [==============================] - 0s 21ms/step - loss: 6.7105 - accuracy: 0.8214
Epoch 9/19
1/1 [==============================] - 0s 21ms/step - loss: 6.6776 - accuracy: 0.8571
Epoch 10/19
1/1 [==============================] - 0s 27ms/step - loss: 6.6424 - accuracy: 0.9286
Epoch 11/19
1/1 [======

In [157]:
def buildPhrase(texts, str_len = 1):
  result = texts
  data = tokenizer.texts_to_sequences([result])[0]
  for i in range(str_len):
    x = keras.utils.to_categorical(data[len(data)-inp_words:], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding

    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict( inp, verbose = "None" ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)

    result += " " + tokenizer.index_word[indx] # дописываем строку

  return result

In [153]:
for i in range(1):
  x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount)

In [154]:
x.shape

(1, 31, 1000)

In [163]:
print(buildPhrase("ответ на проблемы", 20))
print(buildPhrase("вы лучший ответ", 8))
print(buildPhrase("верьте и думайте", 8))

ответ на проблемы которые возникли в понедельник думайте позитивно и верьте в свою способность достигать отличных результатов если вы смогли в понедельник герой
вы лучший ответ понедельник проблемы которые и в понедельник герой позитивно
верьте и думайте способность результатов верьте результатов смогли способность понедельник подняться
